In [206]:
import numpy as np
import pandas as pd
from random import randint

from sklearn.linear_model import LinearRegression

Задаём зависимость цены от всех 4-х параметров

In [322]:
# Создаём сэмпл
n_samples = 1000

age_owner = np.random.choice(80, n_samples) + 21  # возраст владельца квартиры
length = np.random.choice(120, n_samples) + 15  # длина квартиры
width = np.random.choice(80, n_samples) + 10  # ширина (чтобы искусственно разложить площадь на составляющие)
flat = np.random.choice(30, n_samples) + 1  # Этаж, на котором находится квартира

price = 100 * length * width + age_owner * flat * randint(1000, 2000)

data = pd.DataFrame({'age_owner': age_owner, 'length': length, 'width': width, 'flat': flat,
                     'price': price})
data.head(5)

,age_owner,length,width,flat,price
0,80,82,86,2,958640
1,32,42,75,26,1632888
2,60,81,66,27,3100680
3,26,22,64,7,429088
4,37,65,84,14,1366512


In [323]:
data.describe()

,age_owner,length,width,flat,price
count,1000.000000,1000.000000,1000.000000,1000.000000,1.000000e+03
mean,60.225000,72.807000,50.795000,15.225000,1.820920e+06
std,23.598416,34.739237,23.099146,8.708744,1.095008e+06
min,21.000000,15.000000,10.000000,1.000000,8.436800e+04
25%,38.000000,41.000000,31.000000,8.000000,9.643740e+05
50%,60.000000,72.000000,51.000000,15.000000,1.618290e+06
75%,81.000000,103.000000,71.000000,23.000000,2.501646e+06
max,100.000000,134.000000,89.000000,30.000000,5.538192e+06


Смотрим ошибки прогнозирования на одних только исходных данных

In [324]:
from sklearn.metrics import mean_absolute_error

X = data[['age_owner', 'length', 'width', 'flat']]
y = data['price']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['age_owner', 'length', 'width', 'flat']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [24253.1490003   4951.28702182  7117.06612213 94468.20441492]
Bias: -1800004.1386313126
Error: 257453.30733587188


In [325]:
y.median()

1618290.0

Предположим, что цена зависит от размеров

In [326]:
data['sqr1'] = data['length'] * data['width']

In [327]:
reg = LinearRegression().fit(data[['age_owner', 'length', 'width', 'flat', 'sqr1']], y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['age_owner', 'length', 'width', 'flat', 'sqr1']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [24064.85866267  -382.48839592  -716.89112983 94364.25306343
   107.83808925]
Bias: -1395737.7205814505
Error: 245773.91986255872


Видно, что вклад у произведения близок к 100%, но его не хватает. Попробуем перемножить остальные параметры

In [328]:
data['sqr2'] = data['age_owner'] * data['length']
data['sqr3'] = data['age_owner'] * data['width']
data['sqr4'] = data['age_owner'] * data['flat']
data['sqr5'] = data['length'] * data['flat']
data['sqr6'] = data['width'] * data['flat']

In [329]:
reg = LinearRegression().fit(data[['age_owner', 'length', 'width', 'flat',
                                   'sqr1', 'sqr2', 'sqr3', 'sqr4', 'sqr5', 'sqr6']], y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['age_owner', 'length', 'width', 'flat',
                                'sqr1', 'sqr2', 'sqr3', 'sqr4', 'sqr5', 'sqr6']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-2.15704562e-11 -4.49063009e-12  1.62675706e-11  3.80511239e-11
  1.00000000e+02  0.00000000e+00  4.54747351e-13  1.58400000e+03
  2.95541369e-13  4.88942220e-13]
Bias: -1.862645149230957e-09
Error: 1.1396186891943215e-09


И мы угадали. Видно, что вклад вносят sqr1 и sqr4

In [330]:
reg = LinearRegression().fit(data[['sqr1', 'sqr4']], y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(data[['sqr1', 'sqr4']])
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [ 100. 1584.]
Bias: -4.656612873077393e-10
Error: 2.440356183797121e-10


Проверим через полиномиальные преобразования для получения новых фич

In [331]:
from sklearn.preprocessing import PolynomialFeatures

In [332]:
poly = PolynomialFeatures(2)

In [333]:
X = poly.fit_transform(data[['age_owner', 'length', 'width', 'flat']])
y = data['price']
print('Features: {}'.format([(i, f) for i, f in enumerate(poly.get_feature_names())]))


Features: [(0, '1'), (1, 'x0'), (2, 'x1'), (3, 'x2'), (4, 'x3'), (5, 'x0^2'), (6, 'x0 x1'), (7, 'x0 x2'), (8, 'x0 x3'), (9, 'x1^2'), (10, 'x1 x2'), (11, 'x1 x3'), (12, 'x2^2'), (13, 'x2 x3'), (14, 'x3^2')]


In [335]:
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format([(i, c) for i, c in enumerate(reg.coef_)]))
print('Bias: {}'.format(reg.intercept_))

pred_values = reg.predict(X)
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [(0, 0.0), (1, -1.1965539670200087e-11), (2, -8.90840179401664e-12), (3, 7.190914530497139e-12), (4, 3.7551156628623517e-11), (5, -4.547473508864641e-13), (6, -1.6342482922482304e-13), (7, 8.526512829121202e-13), (8, 1584.0000000000014), (9, 4.987121826616203e-13), (10, 99.99999999999915), (11, -1.0729195309977513e-12), (12, -2.7000623958883807e-13), (13, 2.517097641430155e-12), (14, -8.810729923425242e-13)]
Bias: -4.656612873077393e-10
Error: 1.399603206664324e-09
